In [1]:
# This is a python block

# Imports

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

sns.set()

In [3]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import sklearn.metrics as metrics

# Cargar data

- cargo

In [4]:
df_data = pd.read_csv('./dataset_clean.csv', index_col=[0])

# Preprocesamiento rápido

- Resuelvo problemas con descriptores que tienen infinitos

In [5]:
filter = df_data['Vacancy energy (log[eV/O atom])'] == -np.inf
df_data['Vacancy energy (log[eV/O atom])'][filter] = np.nan

<ipython-input-5-4a44a552ca71>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_data['Vacancy energy (log[eV/O atom])'][filter] = np.nan


In [6]:
filter = df_data['Stability (log[eV/atom])'] == -np.inf
df_data['Stability (log[eV/atom])'][filter] = np.nan

<ipython-input-6-b29a07dbe167>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_data['Stability (log[eV/atom])'][filter] = np.nan


- Elimino nulos sí o sí (la otra alternativa sería imputar)

In [7]:
df_data.isnull().sum()

Lowest distortion                     0
A                                     0
B                                     0
Radius A [ang]                        0
Radius B [ang]                        0
Formation energy [eV/atom]            0
Stability [eV/atom]                   0
Volume per atom [A^3/atom]            0
Band gap [eV]                         0
a [ang]                               0
Vacancy energy [eV/O atom]          349
Stability (log[eV/atom])            290
Vacancy energy (log[eV/O atom])    3169
Magnetic moment (^2[mu_B])          947
Valence B_1                           0
Valence B_2                           0
Valence B_3                           0
Valence B_4                           0
Valence B_5                           0
Valence B_element not in BV           0
Valence B_not balanced                0
dtype: int64

In [8]:
df_data.drop('Vacancy energy (log[eV/O atom])', axis=1, inplace= True)

In [9]:
df_data.dropna(inplace=True)

# Split target y patrón

- Split

In [10]:
x = df_data.drop(['Lowest distortion', 'a [ang]'], axis=1)
y = df_data['a [ang]']

- Validación

In [11]:
print(x.shape)
print(y.shape)

(3720, 18)
(3720,)


# Split train y test

- Reducimos la dimensionalidad

In [12]:
cols_str = list(list(x))[2:10]
cols_str.remove('Band gap [eV]')
cols_str

['Radius A [ang]',
 'Radius B [ang]',
 'Formation energy [eV/atom]',
 'Stability [eV/atom]',
 'Volume per atom [A^3/atom]',
 'Vacancy energy [eV/O atom]',
 'Stability (log[eV/atom])']

In [13]:
X_cols = cols_str
y_col = ['a [ang]']

x = df_data[X_cols]
y = df_data[y_col]

- Split

In [14]:
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=12, shuffle = True)

In [15]:
X_test_copy = X_test.copy()
y_test_copy = y_test.copy()

- Validación

In [16]:
print(X_train.shape)
print(X_test.shape)

(2976, 7)
(744, 7)


In [17]:
prubea = X_train.head(1)
prubea

,Radius A [ang],Radius B [ang],Formation energy [eV/atom],Stability [eV/atom],Volume per atom [A^3/atom],Vacancy energy [eV/O atom],Stability (log[eV/atom])
430,0.27,0.96,-0.539,2.738,15.502,-5.142,1.453122


# Escalamiento/Normalización de los datos

- Instancio scaler para target y patron

In [18]:
x_st_scaler = StandardScaler() #Creo instancia del scaler
y_st_scaler = StandardScaler() #Creo instancia del scaler

- Escalo con fit sobre train, y trasnformo test

In [19]:
X_train = x_st_scaler.fit_transform(X_train) # escalo train
X_test = x_st_scaler.transform(X_test) #transformo test

In [20]:
j = x_st_scaler.transform(prubea)
j[0]

array([-2.19355722,  0.75628554,  1.37451068,  3.44472639,  0.61939533,
       -0.96801075,  1.45987294])

In [21]:
y_train = y_st_scaler.fit_transform(y_train.values.reshape(-1,1)) # escalo train
y_test = y_st_scaler.transform(y_test.values.reshape(-1,1)) #transformo test

# Entrenamiento del modelo con Linear Regression

- Importo modelo de Sklearn

In [22]:
from sklearn.ensemble import HistGradientBoostingRegressor

- Instancio modelo

In [23]:
model = HistGradientBoostingRegressor()
#model = HistGradientBoostingRegressor(max_bins= 255,max_iter= 1000,learning_rate= 0.1,max_depth= 8)

- Entreno

In [24]:
model.fit(X_train, y_train)

/usr/local/lib/python3.10/dist-packages/sklearn/utils/validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


HistGradientBoostingRegressor()

- Predigo

In [25]:
y_pred = model.predict(X_test)

- Evaluación rápida

In [26]:
mae = metrics.mean_absolute_error(y_test, y_pred)
mse = metrics.mean_squared_error(y_test, y_pred)
rmse = metrics.mean_squared_error(y_test, y_pred, squared=False)

r2 = metrics.r2_score(y_test, y_pred)

In [27]:
print("mae: \t", mae.round(4))
print("mse: \t", mse.round(4))
print("rmse: \t", rmse.round(4))
print("r2: \t", r2.round(4))

mae: 	 0.1438
mse: 	 0.1062
rmse: 	 0.3258
r2: 	 0.898


In [28]:
a = np.array(X_train)
a

array([[-2.19355722,  0.75628554,  1.37451068, ...,  0.61939533,
        -0.96801075,  1.45987294],
       [-1.06089353, -0.37616878,  1.39870786, ..., -1.22124776,
        -1.23681915,  0.88887139],
       [-0.25184803,  0.89784233, -0.72844398, ...,  0.57276371,
        -0.27685748,  0.8621197 ],
       ...,
       [-0.09003893,  0.89784233, -0.10921622, ...,  0.94469747,
        -0.20654639,  0.295767  ],
       [-0.83436079,  0.47317196,  0.68269139, ...,  0.67908379,
        -0.76066477,  1.06268486],
       [-1.15797899, -0.84802474,  1.41850555, ..., -1.12276179,
        -0.93452928,  0.6418786 ]])

# Deploy

> Básicos

- Imports

In [29]:
from ipywidgets import Dropdown, FloatSlider, IntSlider, interact, FloatText

- Revisando las columnas necesarias/

In [30]:
df_raw=pd.read_csv("./dataset.csv")

> Pipeline

- Función para sacar radios a partir de elementos

In [31]:
def get_radius(elementA, elementB):

    filterA = df_raw['A'] == elementA
    radiusA = df_raw[filterA]['Radius A [ang]'].median()

    filterB = df_raw['B'] == elementB
    radiusB = df_raw[filterB]['Radius B [ang]'].median()

    return[radiusA, radiusB]

- Función para crear df con datos a escalar

In [32]:
def df_creation (data_values):
    data_df = pd.DataFrame(columns=cols_str)
    data_df.loc[0] = data_values
    return(data_df)

In [33]:
# data_values = np.array(list(test.values[0]))
# d = df_creation(data_values)

- Pipeline

In [34]:
def make_predictions(elementA, elementB, formationEnergy, stability, volume, vacancyEnergy):

    radius = get_radius(elementA, elementB) #obtengo radios

    data = radius + [formationEnergy, stability, volume, vacancyEnergy, np.log2(stability)] #concateno los datos

    data_df = df_creation(data) #paso a df

    s_data = x_st_scaler.transform(data_df)#escalo data

    prediction = model.predict(s_data) #Predigo

    prediction = np.array(prediction) #cambio dimenciones para quitar transformacion a la prediccion
    prediction = np.expand_dims(prediction,axis=0)
    prediction = y_st_scaler.inverse_transform(prediction)#quito la transformacion a prediction

    #imprimo resultados
    return f"Predictd 'a' Lattice Parameter:    {prediction[0][0]}"

- Deploy

In [35]:
interact(
    #funcion a ejecutar con la data que recibe
    make_predictions,
    elementA = Dropdown(options =  list(df_raw['A'].unique()) ),
    elementB = Dropdown(options =  list(df_raw['B'].unique()) ),
    formationEnergy = FloatSlider(min = x['Formation energy [eV/atom]'].min(), max= x['Formation energy [eV/atom]'].max(), step = 0.001, value = x['Formation energy [eV/atom]'].mean()),
    # variables, recibidas de sliders, le digo que por defecto reciba la medi
    stability = FloatSlider(min = x['Stability [eV/atom]'].min(), max= x['Stability [eV/atom]'].max(), step = 0.001, value = x['Stability [eV/atom]'].mean()),
    volume = FloatSlider(min = x['Volume per atom [A^3/atom]'].min(), max= x['Volume per atom [A^3/atom]'].max(), step = 0.001, value = x['Volume per atom [A^3/atom]'].mean()),
    vacancyEnergy = FloatSlider(min = x['Vacancy energy [eV/O atom]'].min(), max= x['Vacancy energy [eV/O atom]'].max(), step = 0.001, value = x['Vacancy energy [eV/O atom]'].mean())
);


interactive(children=(Dropdown(description='elementA', options=('Ac', 'Ag', 'Al', 'As', 'Au', 'B', 'Ba', 'Be',…

In [36]:
interact(
    #funcion a ejecutar con la data que recibe
    make_predictions,
    elementA = Dropdown(options =  list(df_raw['A'].unique()) ),
    elementB = Dropdown(options =  list(df_raw['B'].unique()) ),
    formationEnergy = FloatText(min = x['Formation energy [eV/atom]'].min(), max= x['Formation energy [eV/atom]'].max(), step = 0.001, value = x['Formation energy [eV/atom]'].mean()),
    # variables, recibidas de sliders, le digo que por defecto reciba la medi
    stability = FloatText(min = x['Stability [eV/atom]'].min(), max= x['Stability [eV/atom]'].max(), step = 0.001, value = x['Stability [eV/atom]'].mean()),
    volume = FloatText(min = x['Volume per atom [A^3/atom]'].min(), max= x['Volume per atom [A^3/atom]'].max(), step = 0.001, value = x['Volume per atom [A^3/atom]'].mean()),
    vacancyEnergy = FloatText(min = x['Vacancy energy [eV/O atom]'].min(), max= x['Vacancy energy [eV/O atom]'].max(), step = 0.001, value = x['Vacancy energy [eV/O atom]'].mean())
);


interactive(children=(Dropdown(description='elementA', options=('Ac', 'Ag', 'Al', 'As', 'Au', 'B', 'Ba', 'Be',…

- Testing data

In [37]:
y_test_copy.head(2)

,a [ang]
4235,5.492
3168,5.815


In [38]:
X_test_copy.head(2)

,Radius A [ang],Radius B [ang],Formation energy [eV/atom],Stability [eV/atom],Volume per atom [A^3/atom],Vacancy energy [eV/O atom],Stability (log[eV/atom])
4235,0.74,0.95,-2.019,0.246,14.157,-11.587,-2.02327
3168,0.76,1.03,-1.869,0.334,14.471,-5.572,-1.58208


In [40]:
df_raw.iloc[4235]

Chemical formula                    TaAgO3
A                                       Ta
B                                       Ag
In literature                        False
Valence A                     not balanced
Valence B                     not balanced
Radius A [ang]                        0.74
Radius B [ang]                        0.95
Lowest distortion             orthorhombic
Formation energy [eV/atom]          -2.019
Stability [eV/atom]                  0.246
Magnetic moment [mu_B]               0.000
Volume per atom [A^3/atom]          14.157
Band gap [eV]                        1.609
a [ang]                              5.492
b [ang]                              5.917
c [ang]                              8.713
alpha [deg]                           90.0
beta [deg]                            90.0
gamma [deg]                           90.0
Vacancy energy [eV/O atom]         -11.587
Name: 4235, dtype: object

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=a7ebb3ed-5c94-423a-a3b7-e9cd0209ff98' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>